Regresión
Las técnicas de aprendizaje automático supervisado consisten en entrenar un modelo para que opere sobre un conjunto de características y prediga una etiqueta utilizando un conjunto de datos que incluye algunos valores de etiquetas ya conocidos. El proceso de formación ajusta las características a las etiquetas conocidas para definir una función general que pueda aplicarse a nuevas características cuyas etiquetas se desconocen y predecirlas. Se puede pensar en esta función de la siguiente manera, en la que y representa la etiqueta que queremos predecir y x representa las características que el modelo utiliza para predecirla.

y=f(x)

En la mayoría de los casos, x es en realidad un vector que consiste en múltiples valores de características, por lo que para ser un poco más precisos, la función podría expresarse así:

y=f([x1,x2,x3,...]) 

El objetivo del entrenamiento del modelo es encontrar una función que realice algún tipo de cálculo con los valores x que produzca el resultado y. Para ello, aplicamos un algoritmo de aprendizaje automático que intenta ajustar los valores x a un cálculo que produzca y de forma razonablemente precisa para todos los casos del conjunto de datos de entrenamiento.

Existen muchos algoritmos de aprendizaje automático para el aprendizaje supervisado, que pueden dividirse en dos tipos:

Algoritmos de regresión: Algoritmos que predicen un valor y que es un valor numérico, como el precio de una casa o el número de operaciones de venta.
Algoritmos de clasificación: Algoritmos que predicen a qué categoría, o clase, pertenece una observación. El valor y en un modelo de clasificación es un vector de valores de probabilidad entre 0 y 1, uno para cada clase, que indica la probabilidad de que la observación pertenezca a cada clase.
En este cuaderno, nos centraremos en la regresión, utilizando un ejemplo basado en un estudio real en el que se recogieron datos de un sistema de bicicletas compartidas y se utilizaron para predecir el número de alquileres en función de la estacionalidad y las condiciones meteorológicas. Utilizaremos una versión simplificada del conjunto de datos de ese estudio.

Explorar los datos

El primer paso en cualquier proyecto de aprendizaje automático es explorar los datos que se utilizarán para entrenar un modelo. El objetivo de esta exploración es intentar comprender las relaciones entre sus atributos; en particular, cualquier correlación aparente entre las características y la etiqueta que el modelo intentará predecir. Esto puede requerir cierto trabajo para detectar y corregir problemas en los datos (como tratar valores perdidos, errores o valores atípicos), derivar nuevas columnas de características transformando o combinando características existentes (un proceso conocido como ingeniería de características), normalizar características numéricas (valores que se pueden medir o contar) para que estén en una escala similar y codificar características categóricas (valores que representan categorías discretas) como indicadores numéricos.

Empecemos cargando los datos del uso compartido de bicicletas como un DataFrame de Pandas y visualizando las primeras filas.

In [ ]:
import pandas as pd

# load the training dataset
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/ml-basics/daily-bike-share.csv
bike_data = pd.read_csv('daily-bike-share.csv')
bike_data.head()

Los datos constan de las siguientes columnas

instant: Un identificador único de fila<br>
dteday: La fecha en la que se observaron los datos; en este caso, los datos se recogieron diariamente, por lo que hay una fila por fecha.<br>
estación: Un valor codificado numéricamente que indica la estación (1:invierno, 2:primavera, 3:verano, 4:otoño)<br>
año: El año del estudio en el que se realizó la observación (el estudio tuvo lugar a lo largo de dos años: el año 0 representa 2011, y el año 1 representa 2012)<br>
mnth: El mes natural en el que se realizó la observación (1:enero ... 12:diciembre)<br>
holiday (vacaciones): Valor binario que indica si la observación se realizó o no en un día festivo)<br>
día de la semana: El día de la semana en que se realizó la observación (0:domingo ... 6:sábado)<br>
día laborable: Valor binario que indica si el día es laborable o no (no fin de semana ni festivo)<br>
tiempo: Un valor categórico que indica la situación meteorológica (1:despejado, 2:niebla/nubes, 3:lluvia ligera/nieve, 4:lluvia fuerte/granizo/nieve/niebla)<br>
temp: La temperatura en grados centígrados (normalizada)<br>
atemp: La temperatura aparente en grados centígrados (normalizada).<br>
hum: El nivel de humedad (normalizado)<br>
windspeed: La velocidad del viento (normalizada)<br>
alquileres: El número de alquileres de bicicletas registrados.<br>
En este conjunto de datos, alquileres representa la etiqueta (el valor y) para cuya predicción debe entrenarse nuestro modelo. Las otras columnas son características potenciales (valores x).

Como se ha mencionado anteriormente, se puede realizar un poco de ingeniería de características para combinar o derivar nuevas características. Por ejemplo, añadamos una nueva columna llamada día al marco de datos extrayendo el componente día de la columna dteday existente. La nueva columna representa el día del mes del 1 al 31.

In [ ]:
bike_data['day'] = pd.DatetimeIndex(bike_data['dteday']).day
bike_data.head(32)

Bien, comencemos nuestro análisis de los datos examinando algunas estadísticas descriptivas clave. Podemos utilizar el método de descripción del marco de datos para generar estos para las características numéricas, así como la columna de la etiqueta de alquiler.

In [ ]:
numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
bike_data[numeric_features + ['rentals']].describe()

Las estadísticas revelan cierta información sobre la distribución de los datos en cada uno de los campos numéricos, incluido el número de observaciones (hay 731 registros), la media, la desviación típica, los valores mínimo y máximo y los valores de cuartil (los valores umbral para el 25%, el 50% -que también es la mediana- y el 75% de los datos). A partir de aquí, podemos ver que el número medio de alquileres diarios es de unos 848; pero hay una desviación estándar comparativamente grande, lo que indica una gran variación en el número de alquileres diarios.

Podemos hacernos una idea más clara de la distribución de los valores de los alquileres visualizando los datos. Los tipos de gráficos más comunes para visualizar distribuciones de datos numéricos son los histogramas y los gráficos de caja, así que vamos a utilizar la biblioteca matplotlib de Python para crear uno de cada uno de ellos para la columna de alquileres.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# This ensures plots are displayed inline in the Jupyter notebook
%matplotlib inline

# Get the label column
label = bike_data['rentals']


# Create a figure for 2 subplots (2 rows, 1 column)
fig, ax = plt.subplots(2, 1, figsize = (9,12))

# Plot the histogram   
ax[0].hist(label, bins=100)
ax[0].set_ylabel('Frequency')

# Add lines for the mean, median, and mode
ax[0].axvline(label.mean(), color='magenta', linestyle='dashed', linewidth=2)
ax[0].axvline(label.median(), color='cyan', linestyle='dashed', linewidth=2)

# Plot the boxplot   
ax[1].boxplot(label, vert=False)
ax[1].set_xlabel('Rentals')

# Add a title to the Figure
fig.suptitle('Rental Distribution')

# Show the figure
fig.show()


Los gráficos muestran que el número de alquileres diarios oscila entre 0 y algo más de 3.400. Sin embargo, la media (y la mediana) del número de alquileres diarios está más cerca del extremo inferior de ese intervalo. Sin embargo, la media (y la mediana) del número de alquileres diarios está más cerca del extremo inferior de ese rango, con la mayoría de los datos entre 0 y alrededor de 2.200 alquileres. Los pocos valores por encima de esta cifra se muestran en el diagrama de caja como pequeños círculos, lo que indica que son valores atípicos, es decir, valores inusualmente altos o bajos más allá del rango típico de la mayoría de los datos.

Podemos hacer el mismo tipo de exploración visual de las características numéricas. Creemos un histograma para cada una de ellas.

In [ ]:
# Plot a histogram for each numeric feature
for col in numeric_features:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    feature = bike_data[col]
    feature.hist(bins=100, ax = ax)
    ax.axvline(feature.mean(), color='magenta', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='cyan', linestyle='dashed', linewidth=2)
    ax.set_title(col)
plt.show()

Las características numéricas parecen tener una distribución más normal, con la media y la mediana más cerca de la mitad del intervalo de valores, coincidiendo con los valores más frecuentes.

Hemos explorado la distribución de los valores numéricos en el conjunto de datos, pero ¿qué ocurre con las características categóricas? No se trata de números continuos en una escala, por lo que no podemos utilizar histogramas, pero podemos trazar un gráfico de barras que muestre el recuento de cada valor discreto para cada categoría.

In [ ]:
import numpy as np

# plot a bar plot for each categorical feature count
categorical_features = ['season','mnth','holiday','weekday','workingday','weathersit', 'day']

for col in categorical_features:
    counts = bike_data[col].value_counts().sort_index()
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    counts.plot.bar(ax = ax, color='steelblue')
    ax.set_title(col + ' counts')
    ax.set_xlabel(col) 
    ax.set_ylabel("Frequency")
plt.show()


Muchas de las características categóricas muestran una distribución más o menos uniforme (lo que significa que hay aproximadamente el mismo número de filas para cada categoría). Las excepciones son

Vacaciones: Hay muchos menos días festivos que no festivos.<br>
laborables: Hay más días laborables que no laborables.<br>
tiempo: La mayoría de los días son de categoría 1 (despejado), siendo la categoría 2 (niebla y nubes) la siguiente más común. Hay relativamente pocos días de categoría 3 (lluvia ligera o nieve) y ninguno de categoría 4 (lluvia intensa, granizo o niebla).<br>
Ahora que sabemos algo sobre la distribución de los datos en nuestras columnas, podemos empezar a buscar relaciones entre las características y la etiqueta de alquiler que queremos poder predecir.

Para las características numéricas, podemos crear gráficos de dispersión que muestren la intersección de los valores de las características y las etiquetas. También podemos calcular el estadístico de correlación para cuantificar la relación aparente..

In [ ]:
for col in numeric_features:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    feature = bike_data[col]
    label = bike_data['rentals']
    correlation = feature.corr(label)
    plt.scatter(x=feature, y=label)
    plt.xlabel(col)
    plt.ylabel('Bike Rentals')
    ax.set_title('rentals vs ' + col + '- correlation: ' + str(correlation))
plt.show()


Los resultados no son concluyentes, pero si se observan detenidamente los gráficos de dispersión de la temperatura y la temperatura atmosférica, se puede ver una vaga tendencia diagonal que muestra que un mayor número de alquileres tiende a coincidir con temperaturas más altas; y un valor de correlación ligeramente superior a 0,5 para ambas características apoya esta observación. Por el contrario, los gráficos correspondientes a la humedad y la velocidad del viento muestran una correlación ligeramente negativa, lo que indica que hay menos alquileres en los días en que la humedad o la velocidad del viento son elevadas.

Ahora vamos a comparar las características categóricas con la etiqueta. Para ello, crearemos gráficos de caja que muestren la distribución del número de alquileres para cada categoría.

In [ ]:
# plot a boxplot for the label by each categorical feature
for col in categorical_features:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    bike_data.boxplot(column = 'rentals', by = col, ax = ax)
    ax.set_title('Label by ' + col)
    ax.set_ylabel("Bike Rentals")
plt.show()

Los gráficos muestran cierta variación en la relación entre los valores de algunas categorías y los alquileres. Por ejemplo, hay una clara diferencia en la distribución de los alquileres en fin de semana (día de la semana 0 ó 6) y los que se producen durante la semana laboral (día de la semana 1 a 5). Del mismo modo, hay diferencias notables en las categorías de días festivos y laborables. Hay una tendencia notable que muestra diferentes distribuciones de alquileres en los meses de primavera y verano en comparación con los meses de invierno y otoño. La categoría weathersit también parece marcar una diferencia en la distribución del alquiler. La característica del día que creamos para el día del mes muestra poca variación, lo que indica que probablemente no es predictiva del número de alquileres.

Entrenar un modelo de regresión

Ahora que hemos explorado los datos, es hora de utilizarlos para entrenar un modelo de regresión que utilice las características que hemos identificado como potencialmente predictivas para predecir la etiqueta de alquiler. Lo primero que tenemos que hacer es separar las características que queremos utilizar para entrenar el modelo de la etiqueta que queremos que prediga.

In [ ]:
# Separate features and labels
X, y = bike_data[['season','mnth', 'holiday','weekday','workingday','weathersit','temp', 'atemp', 'hum', 'windspeed']].values, bike_data['rentals'].values
print('Features:',X[:10], '\nLabels:', y[:10], sep='\n')

Después de separar el conjunto de datos, ahora tenemos matrices numpy llamadas X que contienen las características e y que contienen las etiquetas.

Podríamos entrenar un modelo utilizando todos los datos, pero es una práctica común en el aprendizaje supervisado dividir los datos en dos subconjuntos: un conjunto (normalmente más grande) con el que entrenar el modelo, y un conjunto más pequeño "retenido" con el que validar el modelo entrenado. Esto nos permite evaluar el rendimiento del modelo cuando se utiliza con el conjunto de datos de validación comparando las etiquetas predichas con las etiquetas conocidas. Es importante dividir los datos aleatoriamente (en lugar de, por ejemplo, tomar el primer 70% de los datos para el entrenamiento y reservar el resto para la validación). Esto ayuda a garantizar que los dos subconjuntos de datos son estadísticamente comparables (de modo que validamos el modelo con datos que tienen una distribución estadística similar a los datos con los que se entrenó).

Para dividir aleatoriamente los datos, utilizaremos la función train_test_split de la biblioteca scikit-learn. Esta biblioteca es uno de los paquetes de aprendizaje automático para Python más utilizados.

In [ ]:
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

Ahora tenemos los siguientes cuatro conjuntos de datos:

X_entrenamiento: Los valores de las características que utilizaremos para entrenar el modelo<br>
y_train: Las etiquetas correspondientes que utilizaremos para entrenar el modelo.<br>
X_prueba: Los valores de las características que utilizaremos para validar el modelo<br>
y_test: Las etiquetas correspondientes que utilizaremos para validar el modelo.<br>
Ahora estamos listos para entrenar un modelo ajustando un algoritmo de regresión adecuado a los datos de entrenamiento. Utilizaremos un algoritmo de regresión lineal, un punto de partida habitual para la regresión, que intenta encontrar una relación lineal entre los valores X y la etiqueta y. El modelo resultante es una función que conceptualiza el valor de la etiqueta y. El modelo resultante es una función que define conceptualmente una línea en la que se cruzan todas las posibles combinaciones de valores X e y.

En Scikit-Learn, los algoritmos de entrenamiento se encapsulan en estimadores, y en este caso utilizaremos el estimador LinearRegression para entrenar un modelo de regresión lineal.

In [ ]:
# Train the model
from sklearn.linear_model import LinearRegression

# Fit a linear regression model on the training set
model = LinearRegression().fit(X_train, y_train)
print (model)

Evaluar el modelo entrenado

Ahora que hemos entrenado el modelo, podemos utilizarlo para predecir los recuentos de alquileres para las características que retuvimos en nuestro conjunto de datos de validación. A continuación, podemos comparar estas predicciones con los valores reales de las etiquetas para evaluar lo bien (¡o no!) que funciona el modelo.

In [ ]:
import numpy as np

predictions = model.predict(X_test)
np.set_printoptions(suppress=True)
print('Predicted labels: ', np.round(predictions)[:10])
print('Actual labels   : ' ,y_test[:10])

Comparar cada predicción con su correspondiente valor real no es una forma muy eficaz de determinar lo bien que predice el modelo. Veamos si podemos obtener una indicación mejor visualizando un gráfico de dispersión que compare las predicciones con las etiquetas reales. También superpondremos una línea de tendencia para tener una idea general de la concordancia entre las etiquetas predichas y las etiquetas reales.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Daily Bike Share Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

Hay una tendencia diagonal definida, y las intersecciones de los valores predichos y reales siguen en general la trayectoria de la línea de tendencia; pero hay bastante diferencia entre la función ideal representada por la línea y los resultados. Esta diferencia representa los residuos del modelo, es decir, la diferencia entre la etiqueta prevista cuando el modelo aplica a los datos de validación los coeficientes aprendidos durante el entrenamiento y el valor real de la etiqueta de validación. Estos residuos, evaluados a partir de los datos de validación, indican el nivel de error esperado cuando el modelo se utiliza con nuevos datos cuya etiqueta se desconoce.

Puede cuantificar los residuos calculando una serie de métricas de evaluación de uso común. Nos centraremos en las tres siguientes:

Error cuadrático medio (ECM): La media de las diferencias al cuadrado entre los valores predichos y los reales. Se trata de una métrica relativa en la que cuanto menor es el valor, mejor es el ajuste del modelo.<br>
Error cuadrático medio (RMSE): La raíz cuadrada del MSE. Proporciona una medida absoluta en la misma unidad que la etiqueta (en este caso, el número de alquileres). Cuanto menor sea el valor, mejor será el modelo (en un sentido simplista, representa el número medio de alquileres en el que las predicciones son erróneas).<br>
Coeficiente de determinación (normalmente conocido como R-cuadrado o R2): Una métrica relativa en la que cuanto mayor es el valor, mejor es el ajuste del modelo. En esencia, esta métrica representa qué parte de la varianza entre los valores predichos y reales de las etiquetas es capaz de explicar el modelo.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_test, predictions)
print("R2:", r2)

Ya hemos cuantificado la capacidad de nuestro modelo para predecir el número de alquileres. Definitivamente tiene cierto poder predictivo, ¡pero probablemente podemos hacerlo mejor!

Resumen
Hemos explorado nuestros datos y ajustado un modelo de regresión básico. En el próximo cuaderno, probaremos otros algoritmos de regresión para mejorar el rendimiento.

Lecturas adicionales
Para obtener más información sobre Scikit-Learn, consulte la documentación de Scikit-Learn.https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics